In [2]:
import open3d as o3d
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import os

from f_extract_surface_patch import *
from f_create_ang_rad_bins import create_ang_rad_bins
from f_helper_functions import *
from f_compare_patches import *
from f_center_embedding import center_embedding

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
os.chdir('c:\\Users\\david\\MT_code\\masif_site_outputs')

# Import the coordinates and the features that have been generated by dMaSIF-site of the following proteins
#   - Subunit A (IgG Fc)
#   - Subunit C (GB1 protein)
#   - Complex (Subunit A and C combined)

coords_A = np.load("1fcc_A_predcoords.npy")
features_A = np.load("1fcc_A_predfeatures_emb1.npy")
features_A = features_A[:, 16:32]

coords_C = np.load("1fcc_C_predcoords.npy")
features_C = np.load("1fcc_C_predfeatures_emb1.npy")
features_C = features_C[:, 16:32]

In [ ]:
# Normalize features
features_A = normalize_m11(features_A)
features_C = normalize_m11(features_C)

In [ ]:
# Parameters

mds = MDS(dissimilarity='precomputed', random_state=0)
max_rho = 12 # diameter of the patches
n_angular_bins = 72
n_radial_bins = 10
n_neighbors = 8

# Create meshgrid n_angular_bins x n_radial_bins (+1 to have the right numb or spaces in between)
angular, radial = np.mgrid[-np.pi:np.pi:complex(n_angular_bins+1), 0:max_rho:complex(n_radial_bins+1)]

# Extract many patches from a surface (coords + center idx + radius)

In [3]:
buried = np.load('buried_C.npy', allow_pickle="TRUE")

In [8]:
random_sample = random.sample(list(buried), k=100)

In [ ]:
for center_C in random_sample:

  # Extract the patch
  patch_indeces, patch_coords, pairwise_distances = extract_surface_patch(coords_C, center_C, 12)
  patch_C = {}
  patch_C["indeces"] = patch_indeces
  patch_C["coords"] = patch_coords
  patch_C["distance_matrix"] = pairwise_distances
  patch_C["features"]=features_C[patch_C["indeces"]]

  #Create embedding using the Geodesic Distances
  embedding_C = mds.fit_transform(patch_C["distance_matrix"])

  features_patch_C = patch_C["features"]

  # Center the embedding
  embedding_C = center_embedding(embedding_C)
  patch_C["embedding_cart"]= embedding_C

  # Convert the embedding to polar coordinates
  polar_coords_C = cart_to_polar(embedding_C)
  patch_C["embedding_polar"] = polar_coords_C

  # To create a feature vector (length 16) for each of the bins and save them in a np.array of shape (angular bins x radial bins x features)
  # Feed the meshgrid matrices, the features matrices, the cartesian coords and a number of nearest neighbors (for the bin feature computation)
  tensor_C = create_ang_rad_bins(angular, radial, features_patch_C, embedding_C, n_neighbors)
  patch_C["feature_tensor"] = tensor_C


  os.chdir('c:\\Users\\david\\MT_code\\extracted_patches\\buried')
  np.save('chain_C_patch_' + str(center_C) + '.npy', patch_C)



# Rotate and compare patches

In [ ]:
tocompare = list(range(0, 7317, 25))

os.chdir('c:\\Users\\david\\MT_code\\extracted_patches\\official')
patch_A = np.load('chain_A_patch_4419.npy', allow_pickle="TRUE").item()
tensor_A = patch_A["feature_tensor"]

best_scores = {p:0 for p in tocompare}

for center_C in tocompare: 
  patch_C = np.load('chain_C_patch_' + str(center_C) + '.npy', allow_pickle="TRUE").item()

  tensor_C = patch_C["feature_tensor"]

  # Give two same-sized tensors, the second one will be rotated and compared with the first
  best_score, best_rotation, best_tensor_C, best_dot_matrix = compare_patches(tensor_A, tensor_C, max_rho)
  print('chain_C_patch_' + str(center_C) + ': ' + str(best_score))

  best_scores[center_C] = best_score

np.save('best_scores_new.npy', best_scores)